In [67]:
import polars as pl

salaries = pl.read_csv('Salaries.csv')
batting = pl.read_csv('Batting.csv')
pitching = pl.read_csv('Pitching.csv')
fielding = pl.read_csv('Fielding.csv')

def filter_and_aggregate(df, cols_to_sum):
    return (
        df.filter(pl.col('yearID') >= 2006)
          .group_by('playerID', 'yearID')
          .agg([pl.col(col).sum() for col in cols_to_sum])
    )

salaries = (
    salaries.filter(pl.col('yearID') >= 2006)
            .group_by('playerID', 'yearID')
            .agg(pl.col('salary').sum())
)

batting_cols = ['stint', 'G', 'G_batting', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
                'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP', 'G_old']
pitching_cols = ['stint', 'W', 'L', 'G', 'GS', 'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER',
                 'HR', 'BB', 'SO', 'BAOpp', 'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP',
                 'GF', 'R', 'SH', 'SF', 'GIDP']
fielding_cols = ['stint', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB',
                 'WP', 'SB', 'CS', 'ZR']

batting = filter_and_aggregate(batting, batting_cols)
pitching = filter_and_aggregate(pitching, pitching_cols)
fielding = filter_and_aggregate(fielding, fielding_cols)

result = (
    salaries
    .join(batting, on=['playerID', 'yearID'], how='left')
    .join(pitching, on=['playerID', 'yearID'], how='left')
    .join(fielding, on=['playerID', 'yearID', 'stint', 'G'], how='left')
    .with_columns(
        Training_Validation=pl.when(pl.col('yearID') == 2016)
                              .then(pl.lit('Validation'))
                              .otherwise(pl.lit('Training'))
    )
)

result

playerID,yearID,salary,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old,stint_right,W,L,G_right,GS,CG,SHO,SV,IPouts,H_right,ER,HR_right,BB_right,SO_right,BAOpp,ERA,IBB_right,WP,HBP_right,BK,BFP,GF,R_right,SH_right,SF_right,GIDP_right,GS_right,InnOuts,PO,A,E,DP,PB,WP_right,SB_right,CS_right,ZR,Training_Validation
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str
"""freemfr01""",2011,414000,1,157,0,571,67,161,32,0,21,76,4,4,53,142,3,6,0,5,15,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""encared01""",2013,8000000,1,142,0,530,90,144,29,1,36,104,7,1,82,62,7,4,0,5,20,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""caminar01""",2015,515500,1,73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,1,5,1,73,0,0,0,0,224,63,30,7,29,73,0.224,3.62,2,6,6,1,318,19,31,1,1,5,0,224,2,9,0,0,null,null,null,null,null,"""Training"""
"""jennija01""",2006,4500000,1,32,0,62,3,8,1,0,0,2,0,0,3,12,0,0,10,0,0,null,1,9,13,32,32,3,2,0,636,206,89,17,85,142,0.258,3.78,7,10,3,0,902,0,94,8,6,21,32,636,14,25,1,3,null,null,null,null,null,"""Training"""
"""kinslia01""",2006,327000,1,120,0,423,65,121,27,1,14,55,11,4,40,64,1,3,1,7,12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""madriwa01""",2010,404730,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""ventejo01""",2011,429500,1,85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,1,6,2,85,0,0,0,5,264,53,18,2,43,96,0.176,1.84,7,4,5,0,357,10,19,7,1,11,0,264,6,19,2,1,null,null,null,null,null,"""Training"""
"""gonzagi01""",2014,8500000,1,27,0,46,3,4,1,0,1,2,0,0,0,26,0,0,6,0,0,null,1,10,10,27,27,0,0,0,476,134,63,10,56,162,0.23,3.57,0,2,3,0,653,0,66,7,4,9,27,476,6,19,1,1,null,null,null,null,null,"""Training"""


In [43]:
duplicates = final_df.group_by(['playerID', 'yearID']).agg([
    pl.count().alias('count')
]).filter(pl.col('count') > 1)

duplicates

/var/folders/j2/qlqp3h0134d5kv_75_ffjz380000gn/T/ipykernel_23551/2113269750.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias('count')


playerID,yearID,count
str,i64,u32
